<a href="https://colab.research.google.com/github/SHermukhammadjon/Atari-Games/blob/main/atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atrai games


## 0. Installing and impr nessery librarys

In [3]:
# ! pip install stable-baselines3[extra]
# ! pip install -q swig
# ! pip install -q gymnasium[box2d]

In [68]:
!apt-get update && apt-get install swig cmake

In [12]:
import gymnasium as gym
from collections import defaultdict

import matplotlib.pyplot  as plt
import matplotlib.patches as patch
import seaborn as sns
import cv2

import numpy as np
import random

from stable_baselines3 import PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv

In [13]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 2. CartPole-v1

### 2.1 testing envorment

In [8]:
env = gym.make('CartPole-v1', render_mode = 'rgb_array')

env.reset()

# Define video writer
width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("test.mp4", fourcc, 30.0, (width, height))


episodes = 10
for episode in range(1, episodes+1):
    observation = env.reset()
    done = False
    score = 0

    while not done:
        action = random.choice([0, 1])
        observation, reward, done, info, _ = env.step(action)
        score += reward

        # Recording envorment
        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        # Write frame to video
        video_writer.write(frame)

    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()

Episode 1 score: 30.0
Episode 2 score: 13.0
Episode 3 score: 22.0
Episode 4 score: 14.0
Episode 5 score: 27.0
Episode 6 score: 16.0
Episode 7 score: 34.0
Episode 8 score: 14.0
Episode 9 score: 12.0
Episode 10 score: 16.0


### 2.2 Treining the model

In [9]:
env = gym.make('CartPole-v1', render_mode = 'rgb_array')
env = DummyVecEnv([lambda : env])

model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = 'training/logs')

Using cpu device


In [22]:
model.learn(total_timesteps = 50_000, log_interval = 5)

Logging to training/logs/PPO_4
------------------------------------------
| time/                   |              |
|    fps                  | 692          |
|    iterations           | 10           |
|    time_elapsed         | 29           |
|    total_timesteps      | 20480        |
| train/                  |              |
|    approx_kl            | 0.0043079434 |
|    clip_fraction        | 0.0145       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.438       |
|    explained_variance   | 0.031        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0119       |
|    n_updates            | 190          |
|    policy_gradient_loss | 0.000202     |
|    value_loss           | 0.2          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 676          |
|    iterations           | 20           |
|    time_elapsed      

### 2.3 Save and reoloding the model

In [12]:
model.save('training/saved_models/OPP_cartpole')

In [13]:
del model

In [20]:
model = PPO.load('training/saved_models/OPP_cartpole', env = env)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(475.8, 72.60000000000001)

### 2.4 Evaluet the model

In [ ]:
evaluate_policy(model, env, n_eval_episodes = 10)

In [ ]:
env.close()

### 2.5 Tesintg the model

In [25]:
env = gym.make('CartPole-v1', render_mode = 'rgb_array')
env = DummyVecEnv([lambda : env])

# Define video writer
width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("CartPole.mp4", fourcc, 30.0, (width, height))


episodes = 10

for episode in range(1, episodes+1):
    observation = env.reset()
    done, score = False, 0

    while not done:
        action, _ = model.predict(observation)
        step = env.step(action)

        observation, reward, done, info = step[0], step[1], step[2], step[3]
        score += reward

        # Recording envorment
        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        # Write frame to video
        video_writer.write(frame)


    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()

Episode 1 score: [500.]
Episode 2 score: [500.]
Episode 3 score: [500.]
Episode 4 score: [500.]
Episode 5 score: [500.]
Episode 6 score: [500.]
Episode 7 score: [500.]
Episode 8 score: [500.]
Episode 9 score: [500.]
Episode 10 score: [500.]


#3. Luna Lander

### 3.1 creat envorment

In [64]:
env = gym.make('LunarLander-v2', render_mode = 'rgb_array')
# env = DummyVecEnv([lambda : env])

### 3.2 Train meodel

In [65]:
model = DQN("MlpPolicy", "LunarLander-v2", verbose=1, exploration_final_eps=0.1, target_update_interval=250)

Using cpu device
Creating environment from the given name 'LunarLander-v2'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [66]:
model.learn(total_timesteps = 100_000, log_interval = 50)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 94       |
|    ep_rew_mean      | -156     |
|    exploration_rate | 0.577    |
| time/               |          |
|    episodes         | 50       |
|    fps              | 3148     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4701     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 92.4     |
|    ep_rew_mean      | -163     |
|    exploration_rate | 0.168    |
| time/               |          |
|    episodes         | 100      |
|    fps              | 3169     |
|    time_elapsed     | 2        |
|    total_timesteps  | 9243     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 90.4     |
|    ep_rew_mean      | -189     |
|    exploration_rate | 0.1      |
| time/               |          |
|    episodes       

### 3.3 Save and reolad the model

In [29]:
model.save('training/saved_models/OPP_luna_lander')

In [30]:
del model

In [31]:
model = PPO.load('training/saved_models/OPP_luna_lander')

In [67]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=212.41 +/- 27.174502007331196


In [68]:
env = gym.make('LunarLander-v2', render_mode = 'rgb_array')
env = DummyVecEnv([lambda : env])

# Define video writer
width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("Luna Lander.mp4", fourcc, 30.0, (width, height))


episodes = 10

for episode in range(1, episodes+1):
    observation = env.reset()
    done, score = False, 0

    while not done:
        action, _ = model.predict(observation)
        step = env.step(action)

        observation, reward, done, info = step[0], step[1], step[2], step[3]
        score += reward

        # Recording envorment
        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        # Write frame to video
        video_writer.write(frame)


    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()

Episode 1 score: [229.35014]
Episode 2 score: [232.67824]
Episode 3 score: [230.20485]
Episode 4 score: [-12.999191]
Episode 5 score: [235.48586]
Episode 6 score: [197.60553]
Episode 7 score: [217.06616]
Episode 8 score: [231.87965]
Episode 9 score: [206.14624]
Episode 10 score: [193.20544]


# 4. BattleZone

In [3]:
env = gym.make('ALE/BattleZone-v5', render_mode = 'rgb_array')

In [4]:
model = DQN("MlpPolicy", 'ALE/BattleZone-v5')

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:231: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 201.62GB > 11.91GB
  warnings.warn(


In [5]:
model.learn(total_timesteps = 50_000)

In [7]:
model.save('training/saved_models/DQN_BattleZone')

In [16]:
del model

In [14]:
model = DQN.load('training/saved_models/DQN_BattleZone')

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:231: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 201.62GB > 1.63GB
  warnings.warn(


In [15]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=0.00 +/- 0.0


In [17]:
env = gym.make('ALE/BattleZone-v5', render_mode = 'rgb_array')
env = DummyVecEnv([lambda : env])

# Define video writer
width, height = 600, 400
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter("BattleZone.mp4", fourcc, 30.0, (width, height))


episodes = 10

for episode in range(1, episodes+1):
    observation = env.reset()
    done, score = False, 0

    while not done:
        action, _ = model.predict(observation)
        step = env.step(action)

        observation, reward, done, info = step[0], step[1], step[2], step[3]
        score += reward

        # Recording envorment
        frame = env.render()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, (width, height))

        # Write frame to video
        video_writer.write(frame)


    print(f"Episode {episode} score: {score}")

video_writer.release()
env.close()
cv2.destroyAllWindows()

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode 1 score: [0.]
Episode 2 score: [0.]
Episode 3 score: [0.]
Episode 4 score: [0.]
Episode 5 score: [0.]
Episode 6 score: [1000.]
Episode 7 score: [0.]
Episode 8 score: [6000.]
Episode 9 score: [6000.]
Episode 10 score: [5000.]
